In [3]:
import numpy as np
import pandas as pd

In [4]:
train_data = pd.read_csv("data/train_data.csv",sep='\t',header=None)
songs = pd.read_csv("data/songs.csv",sep='\t',header=None)
users = pd.read_csv("data/users.csv",sep='\t',header=None)
test_data = pd.read_csv("data/test_data.csv",sep='\t',header=None)

In [5]:
train_data = train_data.rename(columns={train_data.columns[0]:"user",train_data.columns[1]:"song",train_data.columns[2]:"number"})
test_data = test_data.rename(columns={test_data.columns[0]:"user",test_data.columns[1]:"song",test_data.columns[2]:"number"})
songs = songs.rename(columns={songs.columns[0]:"song"})
users = users.rename(columns={users.columns[0]:"user"})


In [6]:
users["user_id"] = ["user_" +str(i) for i in range(5000)]
songs["song_id"] = ["song_" +str(i) for i in range(49564)]

In [7]:
max(train_data.number)

463

In [8]:
def transformation(data, users, songs):
    tmp =pd.merge(data,users,how='left',left_on=data.user, right_on=users.user)
    tmp1 = pd.merge(tmp,songs,how='left',left_on=tmp.song, right_on=songs.song)
    return tmp1[tmp1.columns[[4,6,2]]]

In [9]:
train_data = transformation(train_data,users,songs)
test_data = transformation(test_data, users, songs)

In [10]:
n,m = len(users),len(songs)

In [11]:
matrix = np.zeros((n,m))

In [12]:
test_data[test_data.user_id=='user_0']

,user_id,song_id,number
64934,user_0,song_6655,1
64935,user_0,song_48270,1
64936,user_0,song_42069,1
64937,user_0,song_45281,1
64938,user_0,song_23084,2
64939,user_0,song_19901,1
64940,user_0,song_21267,1
64941,user_0,song_37714,4
64942,user_0,song_7279,1
64943,user_0,song_13374,1


In [13]:
train_data[train_data.user_id=="user_0"]

,user_id,song_id,number
62601,user_0,song_11814,1
62602,user_0,song_36979,1
62603,user_0,song_32134,2
62604,user_0,song_9733,1
62605,user_0,song_2889,1
62606,user_0,song_23812,2
62607,user_0,song_33351,1
62608,user_0,song_11195,2
62609,user_0,song_22433,1
62610,user_0,song_14362,1


In [14]:
def to_matrix(data,matrix):
    for r in train_data.iterrows():
        tmp = r[1]
        row = int(tmp[0].split("_")[-1])
        col = int(tmp[1].split("_")[-1])
        number = tmp[2]
        matrix[row][col] = number
    return matrix

In [15]:
V_train = to_matrix(train_data,matrix)

In [16]:
V_test = to_matrix(test_data, matrix)

In [74]:
def indicate(matrix):
    index_row,index_col = np.nonzero(matrix)
    M = np.zeros(matrix.shape)
    for i in range(len(index_row)):
        M[index_row[i]][index_col[i]] = 1
    return M
def normalization(matrix, max_rating ):
    matrix +=1
    matrix[matrix>max_rating+1] = max_rating
    matrix[matrix<1] = 1
    matrix[matrix<0] = 1
    return matrix
def SVD_incremental(V_train,feature_number,iteration_number,regu = 0.001,max_rating =483, learning_rate = 0.006):
    n,m = V_train.shape
    U = np.ones((n, feature_number))*0.01
    M = np.ones((m, feature_number))*0.01
    I = indicate(V_train)
    for i in range(iteration_number):
        MUL = np.dot(U,M.T)
        MUL = normalization(MUL, max_rating)
        tmp = np.multiply((V_train - MUL),I)
        delta_U = []
        delta_M = []
        for i in range(n):
            delta_U.append(np.sum(tmp[i]*M.T,axis=1))
        delta_U= np.array(delta_U) - regu*U

        for i in range(m):
            delta_M.append(np.sum(tmp.T[i]*U.T,axis=1))
        delta_M= np.array(delta_M) - regu*M
        
        
        U = U + delta_U*learning_rate
        M = M + delta_M *learning_rate
        print error(V_train,U,M,I,regu)
    return U,M

In [75]:
SVD_incremental(V_train,feature_number=10,iteration_number=100)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [47]:
def error(V_train, U,M,I,regu):
    MUL = V_train-np.dot(U,M.T)
    error = np.sum(I*(MUL**2))
    error = error 
    error = error * 0.5
    return error 

In [70]:
v= np.array([[1,3,2,0],[3,1,5,0],[4,5,3,7],[0,2,3,0],[0,0,4,0]])
v.shape

(5, 4)

In [43]:
SVD_incremental(V_train,feature_number=3,iteration_number=10)

KeyboardInterrupt: 